In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
# import sasc.notebook_helper
import sasc.viz
import scipy.special
from collections import defaultdict
from pprint import pprint
import sasc.analyze_helper
import joblib
# import viz
from sasc.config import RESULTS_DIR

setting = 'default'
MAIN_DIR = join(RESULTS_DIR, 'stories', setting)
EXPT_DIRS = sorted([join(MAIN_DIR, dir_name) for dir_name in os.listdir(MAIN_DIR)])
EXPT_DIRS = [x for x in EXPT_DIRS if 'uts03' in x.lower() or 'uts01' in x.lower()]
EXPT_DIR = EXPT_DIRS[0]
rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
expls = rows.expl.values
try:
    prompts_paragraphs = joblib.load(
        join(EXPT_DIR, "prompts_paragraphs.pkl"),
    )
    prompts = prompts_paragraphs["prompts"]
    paragraphs = prompts_paragraphs["paragraphs"]
except:
    prompts = open(join(EXPT_DIR, "prompts.txt"), 'r').read().split('\n\n')
    paragraphs = open(join(EXPT_DIR, "story.txt"), 'r').read().split('\n\n')
    assert len(prompts) == len(paragraphs), f"{len(prompts)} != {len(paragraphs)}"
# voxel_nums = rows.module_num.values
# subjects = rows.subject.values

### Heatmaps

**Module** <> Story Heatmap

In [ ]:
res = {}
for EXPT_DIR in EXPT_DIRS:
    scores_data = joblib.load(
        join(EXPT_DIR, f"scores_mod_ngram_length={0}.pkl"))
    s = scores_data["scores_mean"].T
    # z-score each column
    s = (s - s.mean(axis=0)) / s.std(axis=0)

    # s = scipy.special.softmax(s, axis=0)
    # s = (s - s.min()) / (s.max() - s.min())
    # rows = joblib.load(join(EXPT_DIR, f'rows1_rep.pkl'))
    rows = joblib.load(join(EXPT_DIR, f"rows.pkl"))
    expls = rows.expl.values

    if setting == 'default':
        expls_order = sasc.analyze_helper.sort_expls_semantically(expls)
        s = s[expls_order][:, expls_order]  # [:, expls_order]
        expls = expls[expls_order]

    sasc.viz.heatmap(s, expls, ylab="Story", xlab="Module")

    # calculate mean of diagonal - mean of off-diagonal for s
    diag_diff = (
        np.mean(np.diag(s))
        - (
            np.mean(s[np.triu_indices_from(s, k=1)])
            + np.mean(s[np.tril_indices_from(s, k=-1)])
        )
        / 2
    ).round(5)
    if not setting == 'default':
        plt.title(os.path.basename(EXPT_DIR) + ' diag_diff=' + str(diag_diff))
    res[os.path.basename(EXPT_DIR)] = diag_diff

    # plt.savefig(join(EXPT_DIR, f"story_module_match.png"), dpi=300)
    plt.savefig(join(EXPT_DIR, f"story_module_match.pdf"), bbox_inches="tight")
    # plt.tight_layout()
    # plt.show()
# sasc.viz.save_figs_to_single_pdf('../results/expl_story_match.pdf')
print(res)

**Data** <> Story Heatmap

In [ ]:
for EXPT_DIR in EXPT_DIRS:
    d = joblib.load(join(EXPT_DIR, "scores_data.pkl"))
    s = d["scores_mean"].T
    # normalize each column
    s = (s - s.mean(axis=0)) / \
        s.std(axis=0)

    rows = joblib.load(join(EXPT_DIR, f"rows.pkl"))
    expls = rows.expl.values
    if setting == 'default':
        expls_order = sasc.analyze_helper.sort_expls_semantically(expls)
        s = s[expls_order][:, expls_order]  # [:, expls_order]
        expls = expls[expls_order]
    # {"scores_mean": scores_mean, "scores_all": scores_all},
    sasc.viz.heatmap(s, expls, ylab="Story", diverging=True,
                     xlab="Explanation", clab='Ngram matching score (normalized)')

    plt.savefig(join(EXPT_DIR, f"story_data_match.pdf"), bbox_inches="tight")

**Module** <> Story Heatmap when varying ngram lengths

In [ ]:
for EXPT_DIR in EXPT_DIRS:
    # keys: scores_mod, scores_max_mod, all_scores, all_ngrams
    # ngram_lengths = [10, 50, 100, 384]
    ngram_lengths = [0]
    for ngram_length in ngram_lengths:
        scores_mod_dict = joblib.load(
            join(EXPT_DIR, f'scores_mod_ngram_length={ngram_length}.pkl'))
        # rows = joblib.load(join(EXPT_DIR, f'rows1_rep.pkl'))
        rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
        expls = rows.expl.values

        s = scores_mod_dict['scores_mean'].T
        # normalize each column
        s = (s - s.mean(axis=0)) / s.std(axis=0)

        if setting == 'default':
            expls_order = sasc.analyze_helper.sort_expls_semantically(expls)
            s = s[expls_order][:, expls_order]  # [:, expls_order]
            expls = expls[expls_order]

        sasc.viz.heatmap(s, expls, ylab='Story',
                         xlab='Module (fit to fMRI voxel)', clab='Mean module response')

        n = s.shape[0]
        # plt.plot([0, n], [0, n], '--', color='gray', alpha=0.1)
        # plt.title(f'Model window: {ngram_length} words', fontsize='small')
        # plt.savefig(
        # join(EXPT_DIR, f'mod_story_match_ngram_length={ngram_length}.png'), dpi=300)
        plt.savefig(join(
            EXPT_DIR, f'mod_story_match_ngram_length={ngram_length}.pdf'), bbox_inches='tight')
        # plt.show()
# sasc.viz.save_figs_to_single_pdf('../results/expl_mod_match.pdf')

### Best voxels table

In [ ]:
vox_diffs = defaultdict(list)
for EXPT_DIR in EXPT_DIRS:
    ngram_length = 0
    scores_mod_dict = joblib.load(
        join(EXPT_DIR, f'scores_mod_ngram_length={ngram_length}.pkl'))
    rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
    expls = rows.expl.values

    s = scores_mod_dict['scores_mean'].T
    s = scipy.special.softmax(s, axis=0)

    BLOCK_SIZE = 4
    for i in range(0, rows.shape[0]):
        row = rows.iloc[i]
        s_row = s[i]
        block_idxs = range((i // BLOCK_SIZE) * BLOCK_SIZE,
                           (i // BLOCK_SIZE + 1) * BLOCK_SIZE)
        diff = s_row[i] - np.mean([s_row[j]
                                  for j in range(s_row.shape[0]) if j not in block_idxs])
        vox_diffs[(row.module_num, row.expl)].append(diff)
vox_diffs = {k: np.mean(v) for k, v in vox_diffs.items()}
vox_diffs

In [ ]:
# display all rows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(vox_diffs.items()).sort_values(
        1, ascending=False).head(30))